In [ ]:
import json
from openslide import OpenSlide
import os
from shapely.geometry import Polygon

In [ ]:
# Load the GeoJSON file
with open('path/to/your_geojson_file.geojson') as f:
    geojson_data = json.load(f)

# Extract the coordinates of the ROI
coordinates = geojson_data['geometry']['coordinates'][0]

# Create a polygon from the coordinates using the shapely library
roi_polygon = Polygon(coordinates)

In [ ]:
# Define the path to the whole-slide image (wsi)
wsi_path = "path/to/your_wsi_image.svs"

# Open the whole-slide image using OpenSlide
slide = OpenSlide(wsi_path)

# Define the size of the tiles to be extracted
tile_size = (256, 256)

# Specify the level of magnification to use
level = 0

# Define the directory to store the output tiles
output_dir = "path/to/output_tiles"

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Initialize an empty list to store the extracted tiles
tiles = []

# Iterate through the slide to extract tiles
for y in range(0, slide.level_dimensions[level][1], tile_size[1]):
    for x in range(0, slide.level_dimensions[level][0], tile_size[0]):
        # Read a region of the slide at the specified level and tile size
        region = slide.read_region((x, y), level, tile_size)

        # Create a bounding box for the tile
        tile_bbox = Polygon(
            [(x, y), (x + tile_size[0], y), (x + tile_size[0], y + tile_size[1]), (x, y + tile_size[1])]
        )

        # Check if the tile intersects with the Region of Interest (ROI) polygon: add it to the list
        if tile_bbox.intersects(roi_polygon):
           tiles.append(region)

# Save the tiles to the output directory
for i, tile in enumerate(tiles):
    # Save each tile as a PNG file with a unique identifier
    tile.save(f"{output_dir}/tile_{i}.png")

# Close the whole-slide image after processing
slide.close()